# Generate ART Risk Assessment Report

This notebook is the fourth step in the demo flow:

1. **01-red-team-prompt-generation.ipynb** — generates adversarial prompts via an sdg_hub flow → saves JSON to `$XDG_DATA_HOME`
2. **02-garak-to-sdg.ipynb** — converts that JSON into garak's expected format (`trait_typology.json` + `intent_stubs/*.txt`) under `$XDG_DATA_HOME/garak/data/cas/`
3. **03-run-garak.ipynb** — runs garak against the target LLM and writes a `.report.jsonl` to `$XDG_DATA_HOME/garak/garak_runs/`
4. **04-generate-report.ipynb** *(this notebook)* — reads the most recent `.report.jsonl` and renders an HTML risk assessment report alongside it

## Setup

In [5]:
import os
from pathlib import Path

from llama_stack_provider_trustyai_garak.utils import _ensure_xdg_vars
from llama_stack_provider_trustyai_garak.result_utils import generate_art_report

_ensure_xdg_vars()

print(f"XDG_DATA_HOME: {os.environ['XDG_DATA_HOME']}")

XDG_DATA_HOME: /Users/hjrnunes/workspace/redhat/hjrnunes/rh-summit-demos/tmp


## Select report

By default, picks the most recent `.report.jsonl` from `$XDG_DATA_HOME/garak/garak_runs/`.
Set `report_path` explicitly to use a specific run.

In [6]:
# Override this to target a specific report, e.g.:
# report_path = Path("/path/to/garak.<UUID>.report.jsonl")
report_path = None

if report_path is None:
    garak_runs = Path(os.environ["XDG_DATA_HOME"]) / "garak" / "garak_runs"
    reports = sorted(garak_runs.glob("*.report.jsonl"))
    if not reports:
        raise FileNotFoundError(f"No .report.jsonl files found in {garak_runs}")
    report_path = reports[-1]

assert report_path.exists(), f"Report not found: {report_path}"
print(f"JSONL report: {report_path.resolve()}")

JSONL report: /Users/hjrnunes/workspace/redhat/hjrnunes/rh-summit-demos/tmp/garak/garak_runs/garak.cad5a346-c1a2-4b57-899a-8b740a19268b.report.jsonl


## Generate HTML report

In [7]:
report_content = report_path.read_text()
html = generate_art_report(report_content)

# Write alongside the JSONL: garak.<UUID>.report.jsonl -> garak.<UUID>.report.html
html_path = report_path.with_name(report_path.name.replace(".jsonl", ".html"))
html_path.write_text(html)

print(f"HTML report:  {html_path.resolve()}")

HTML report:  /Users/hjrnunes/workspace/redhat/hjrnunes/rh-summit-demos/tmp/garak/garak_runs/garak.cad5a346-c1a2-4b57-899a-8b740a19268b.report.html
